In [1]:
import pandas as pd
import pickle
import os
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
import yaml
import cv2
import numpy as np
import matplotlib.pyplot as plt

class DataSet():
    def __init__(self , config):
        
        self.config= config
        self.vid_path = os.path.join('..',config['dataset']['path'],'video')
        
        
        
        with open ('../../../data/whole_train_dataset.pandas','rb') as file:
            self.df_dataset  = pickle.load(file)
            
        vids = "../../../data/video"
        vids =  os.listdir(vids)
        self.df_dataset  =self.df_dataset .loc[self.df_dataset ['filename'].isin(vids)].reset_index(drop=True)
        
#     def split_train_test(self):
#         return train_test_split(self.df_dataset, self.df_dataset.stalled, test_size=0.15, random_state=42,
#                                 stratify=self.df_dataset)
    
    def split_train_test(self):
        return train_test_split(self.df_dataset, self.df_dataset.stalled, test_size=0.15, random_state=42)    
    
    def preprocess_standard(self, x):
        return x / 255.  
    
#     def prepare_image(self, img, size, preprocessing_function, aug=False):
#         img = scipy.misc.imresize(img, size)
#         img = np.array(img).astype(np.float64)
#         if aug: img = augment(img, np.random.randint(7))
#         img = preprocessing_function(img)
#         return img
    @staticmethod
    def getFrame( vidcap , sec , image_name ):
        vidcap.set(cv2.CAP_PROP_POS_MSEC, sec * 1000)
        hasFrames,image = vidcap.read()
        if(hasFrames):
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image ,hasFrames
    
    
    @staticmethod
    def extract_location_area_from_highlighted_curve(image , size,  preprocessing_function):
        # convert to hsv to detect the outlined orange area
        hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
        lower_red = np.array([100,120,150])
        upper_red = np.array([110,255,255])
        # create a mask
        mask1 = cv2.inRange(hsv, lower_red, upper_red)
        mask1 = cv2.dilate(mask1, None, iterations=2)
        mask_ind = np.where(mask1>0)
        xmin , xmax = min(mask_ind[1]) , max(mask_ind[1])
        ymin , ymax = min(mask_ind[0]) , max(mask_ind[0])
        # remove orange line from the image
        image[mask_ind ]=0,0,0
        # fill the area to skip the data outside of this area
        ret,mask1 = cv2.threshold(mask1,10,255,cv2.THRESH_BINARY_INV)
        contours,hierarchy = cv2.findContours(mask1, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        contours = [ctr for ctr in contours if cv2.contourArea(ctr) < 5*(mask1.shape[0]*mask1.shape[1])/6]
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        cv2.drawContours(mask1, [contours[-1]], -1, (0, 0, 0), -1)
        # remove data out of the outlined area
        image[mask1>0] = (0,0,0)

        image = image[ ymin:ymax , xmin:xmax ]


        mask2 = cv2.cvtColor(image , cv2.COLOR_RGB2GRAY)
        ret,mask2 = cv2.threshold(mask2,90,255,cv2.THRESH_BINARY)

        contours,hierarchy = cv2.findContours(mask2, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        # contours = [ctr for ctr in contours if cv2.contourArea(ctr) < 5*(mask1.shape[0]*mask1.shape[1])/6]
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        

        vessels_image  = np.zeros_like(image)


        areas = []
        for ctr in contours:
            if cv2.contourArea(ctr) > 50:

                cv2.drawContours(image, [ctr], -1, (255, 0, 0), -1)
                cv2.drawContours(vessels_image, [ctr], -1, (255, 255, 255), -1)

                xxmin , xxmax = min(ctr[:,:,0])[0] , max(ctr[:,:,0])[0]
                yymin , yymax = min(ctr[:,:,1])[0] , max(ctr[:,:,1])[0]
    #             image = cv2.rectangle(image , (xxmin ,yymin) ,(xxmax , yymax),(0,255,0),1,1)
                areas.append([xxmin ,yymin ,xxmax ,yymax ,cv2.contourArea(ctr) ])
    #             print(xxmin ,yymin ,xxmax ,yymax)
#         plt.figure()
#         plt.imshow(np.hstack((image,vessels_image)))
        vessels_image = cv2.resize(vessels_image ,(size[0], size[1]))
        vessels_image = cv2.cvtColor(vessels_image , cv2.COLOR_RGB2GRAY)
        vessels_image = preprocessing_function(vessels_image)
    #     area
        return vessels_image.astype(np.float64) 


    
    def process_video(self, filename, size, preprocessing_function):
        
        vidcap = cv2.VideoCapture(os.path.join(self.vid_path,filename))
        total_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
        frame_size = (int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)) , int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT )))
        fps = vidcap.get(cv2.CAP_PROP_FPS)
        Video_len = total_frames / fps
        from_sec = 0
        step  = 1.
        time_stamp = np.linspace(from_sec , Video_len , int(total_frames / step) )

        vessels_tensor = np.zeros([1,size[0],size[1]])

        for frame in range(int(total_frames)):
            image , hasframe = self.getFrame(vidcap ,time_stamp[frame] , frame)
            if hasframe:
                vessels_image = self.extract_location_area_from_highlighted_curve(image ,size, preprocessing_function)
                vessels_tensor =np.append(vessels_tensor ,vessels_image[np.newaxis,...], axis=0)
                
        if vessels_tensor.shape[0]<99:
            vessels_tensor = np.append(vessels_tensor , np.zeros((100 - len(vessels_tensor),size[0], size[1])),axis=0)


        return vessels_tensor[...,np.newaxis]
    
    def get_class_one_hot(self, tag , min_value= 0):
        
        onHotTarget = np.ones((2), dtype=np.float64 )* min_value
        onHotTarget[tag] = 1
        return onHotTarget
 
    def data_generator(self, data, which_net='standard', size=(224,224), batch_size=2): 
        if which_net == 'resnet50': 
            preprocessing_function=self.preprocess_input_resnet50
        elif which_net == 'densenet': 
            preprocessing_function=self.preprocess_input_densenet
        elif which_net == 'inception': 
            preprocessing_function=self.preprocess_input_inception
        elif which_net == 'vgg': 
            preprocessing_function=self.preprocess_input_vgg16
        elif which_net == 'standard': 
            preprocessing_function=self.preprocess_standard
            
#         filename, tag = data.loc[0,['filename','stalled']].values
#         processed_video = self.process_video(filename, size, preprocessing_function)
#         return self.process_video(filename, size, preprocessing_function)
        
        while True:
            for start in range(0, len(data), batch_size):
                x_batch = []
                y_batch = []
                end = min(start + batch_size, len(data))
                data_batch = data[start:end]
                for filename, tag in data_batch.loc[:,['filename','stalled']].values:
                    processed_video = self.process_video(filename, size, preprocessing_function)
                    x_batch.append(processed_video)
                    y_batch.append(self.get_class_one_hot(tag))
                x_batch = np.array(x_batch)
                y_batch = np.array(y_batch)
                yield x_batch, y_batch
#             return x_batch, y_batch
                
                
config = '../../script/config.yml'   

with open (config , 'rb') as file:
    config = yaml.safe_load(file)
    
# print(config)
# dataset = DataSet(config)

In [141]:
# k =dataset.data_generator(dataset.df_dataset)
# k[0].shape ,k[1].shape
# dataset.df_dataset.loc[0,['filename','stalled']].values

# dataset.df_dataset.stalled

In [142]:
# with open ('../../../data/whole_train_dataset.pandas','rb') as file:
#             df_dataset  = pickle.load(file)
# df_dataset[1:10].loc[:,['filename','vid_id']].values
# df_dataset

In [2]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import *
from keras.layers import LSTM
from keras.datasets import imdb


model = Sequential()
model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2), activation='relu', padding='same'), input_shape=(100, 224, 224, 1)))
model.add(TimeDistributed(Conv2D(32, (3,3), kernel_initializer="he_normal", activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
model.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
model.add(TimeDistributed(Flatten()))
 
model.add(Dropout(0.5))
model.add(LSTM(256, return_sequences=False, dropout=0.5))
model.add(Dense(2, activation='softmax'))
model.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 100, 112, 112, 32) 1600      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 100, 110, 110, 32) 9248      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 100, 55, 55, 32)   0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 100, 55, 55, 64)   18496     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 100, 55, 55, 64)   36928     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 100, 27, 27, 64)   0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 100, 27, 27, 128) 

In [3]:
data = DataSet(config)
X_train, X_test, y_train, y_test = data.split_train_test()

In [ ]:
from keras.utils import multi_gpu_model
from keras.optimizers import Adam, SGD

# parallel_model = multi_gpu_model(model, gpus=4)
# parallel_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics = ['accuracy'])

batch_size = 2
epochs = 1
size = (224, 224)

train_steps = len(X_train) / batch_size
valid_steps = len(X_test) / batch_size
        
# parallel_model.fit_generator(data.data_generator(X_train, 'standard', size=size, batch_size=batch_size), 
#                     train_steps, epochs=epochs, verbose=5, 
#                     validation_data=data.data_generator(X_test, 'standard', size=size, batch_size=batch_size), 
#                     validation_steps=valid_steps)

model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics = ['accuracy'])

model.fit_generator(data.data_generator(X_train, 'standard', size=size, batch_size=batch_size), 
                    train_steps, epochs=epochs, verbose=2, 
                    validation_data=data.data_generator(X_test, 'standard', size=size, batch_size=batch_size), 
                    validation_steps=valid_steps)

Epoch 1/1
